# Import libraries

In [ ]:
import os
import sys
sys.path.insert(0, "../src")

import gcsfs
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay, precision_recall_curve
import torch
from tqdm.notebook import trange

from dataloader.data_loader import AttributesDataset, get_transform
from model.net import avg_acc_gpu, avg_f1_score_gpu, confusion_matrix, Net
from utils.utils import load_checkpoint, Params

rng = default_rng()

%load_ext tensorboard

In [ ]:
os.environ["TENSORBOARD_BINARY"] = "/opt/conda/envs/cv3d-env/bin/tensorboard"

In [ ]:
gfs = gcsfs.GCSFileSystem(project="airesearch-1409")

# Define variables

In [ ]:
root = "gs://hm_images/"
model_path = "../experiments/local_test"
img_path = "images"
annotation_path = "annotations"

thr = 0.5

In [ ]:
params = Params(
    {
        "num_classes":90,
        "dropout": 0.5,
        "height": 232,
        "width": 232,
        "crop": 224,
        "data_dir": root,
        "batch_size": 128,
        "cuda": torch.cuda.is_available(),
        "device": "cuda:0",
    }
)

# Load dataset

In [ ]:
transform = get_transform(False, params)

test_ds = AttributesDataset(params.data_dir, "annotations/test.csv", transform, gfs)

In [ ]:
start = rng.integers(len(test_ds) - params.batch_size - 1)
inp_data, labels = [], []
for i in trange(start, start + params.batch_size):
    img, label = test_ds[i]
    inp_data.append(img)
    labels.append(label)
inp_data = torch.stack(inp_data, 0)
labels = torch.stack(labels, 0)

print(inp_data.shape)
print(labels.shape)

In [ ]:
data = pd.read_csv(os.path.join(root, "annotations/test.csv"))
cols = data.columns.tolist()[1:]
cols

# Load model

In [ ]:
model = Net(params)
load_checkpoint(os.path.join(model_path, "best.pth.tar"), model);

In [ ]:
%tensorboard --logdir ../experiments/local_test/logs

# Prediction

In [ ]:
model.eval()
if params.cuda:
    model.to(params.device)

with torch.no_grad():
    if params.cuda:
        inp_data = inp_data.to(params.device)
        labels = labels.to(params.device)
    output = model(inp_data)

In [ ]:
labels_cpu = labels.cpu().numpy()
preds = torch.sigmoid(output).cpu().numpy()

In [ ]:
mat = confusion_matrix(output, labels, thr).numpy()

print(f"Avg. Accuracy: {avg_acc_gpu(output, labels, thr):.3f} @ {thr}")
print(f"Avg. F1 score: {avg_f1_score_gpu(output, labels, thr):.3f} @ {thr}")

In [ ]:
idx = 0

fig, ax = plt.subplots(1, 2, figsize=(15, 6))
ConfusionMatrixDisplay(mat[idx]).plot(ax=ax[0], cmap="Blues");

prec, recall, _ = precision_recall_curve(
    labels_cpu[:, idx], preds[:, idx]
)
PrecisionRecallDisplay(prec, recall).plot(ax=ax[1])
fig.suptitle(f"{cols[idx]}", fontsize=16)
fig.tight_layout()

# Visualize

In [ ]:
test_imgs = inp_data.cpu().numpy() * np.asarray([0.229, 0.224, 0.225]).reshape(1, -1, 1, 1)
test_imgs += np.asarray([0.485, 0.456, 0.406]).reshape(1, -1, 1, 1)
test_imgs = test_imgs.clip(0.0, 1.0)

col_names = np.asarray(cols)

In [ ]:
i = rng.integers(params.batch_size)

plt.imshow(test_imgs[i, ...].transpose(1, 2, 0))
print(f"Labels: {col_names[labels_cpu[i].astype(bool)]}")
print(f"Predictions: {col_names[preds[i] > thr]}")
print(f"Scores: {preds[i, preds[i] > thr]}")